TODO
--
* [X] Cómo cargar archivo en layout
* [X] Subir instancias al repositorio
* [X] Algoritmo general que seleccione stack usando heurística, 
* [X] desmantela lo necesario USANDO SDPP
* [ ] vuelve a llenar. usando `SFpp` (Pendiente, esta explotando en algunas instancias)
* [ ] Arreglar rank y se genere usando contenedores que se van a extraer y para que acepte repetidos 
* [ ] Probar usando instancias y revisar *posibles errores*

Algunos problemas conocidos (de las funciones de arriba)
----

- `SDpp` desmantela completamente el stack [CHECK]
- `SFpp` tiene errores
- Puede que tenga problemas si hay elementos con el mismo valor (rank es un mapa por lo que considerará uno de los elementos)
- Si alguna función no puede hacer su trabajo fallará
- El tema de la holgura (slack) hay que revisarlo. Es para instancias donde hay *espacio de sobra* para colocar elmentos

Y aqui va lo mio supongo :p
----
<a rel="pixie">![Foo](https://images-na.ssl-images-amazon.com/images/I/61mBskokGZL._AC_SL1280_.jpg)</a>

Cosas mias que no se porque funcionan pero funcionan
--

* La función `read(path)` recibe como parametro `path` y retorna un layout.
* La función `select_col(layout)` selecciona una columna para trabajar (la que se re-estructurara) y omite las columnas ordenadas, retorna el indice de la columna seleccionada o un -1 en caso de que todo este ordenado.
* La función `capac(layout,column)` a.k.a lo que hice en java pero aca para ver el espacio disponible dinamico.
* La funcion `remove_until(column)` indica cuantos elementos hay que sacar de la columna para reordenar, en caso que algunos contenedores ya se encuentren ordenados en la base del stack.


Lectura de archivo
--


In [10]:
import Layout
import Greedy


flag = 0
#Columnas
columns = 0
#Total contenedores
total = 0
maxSize = 0
stacks = []
with open("C:\\Users\\Leviathan\\Desktop\\trabajo\\Referencias\\BF\\test\\10-6\\data10-6-17.dat") as file:
    for line in file:
        s = line.split()
        #Si es la primera linea entonces se guarda las columnas y el total
        if flag == 0:
            #Actualiza columns
            columns = s[0]
            #Actualiza total
            total = s[1]
            #Actualiza flag para que no pase denuevo
            flag=1
        #En caso contrario sen otros datos de interes
        else:
            #Se ve el size mayor de la columna para definirlo automatico
            aux=int(s[0])
            if maxSize <= aux:
                maxSize = aux;
            
            #Se crean los stacks
            temporalStack = []
            for i in range(1,len(s)):
                temporalStack.append(int(s[i]))
            stacks.append(temporalStack)

#Una vez leido el archivo imprimo los stacks para ver si quedo bien
#for s in stacks:
#  print (s)
#maxSize += 2
#print (maxSize)



FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\Leviathan\\Desktop\\trabajo\\Referencias\\BF\\test\\10-6\\data10-6-17.dat'

In [11]:
#Se crea el layout
maxSize += 2
layout = Layout.Layout(stacks, maxSize)
#move c_o->c_d
#layout.move(0,1)

print (maxSize)





2


Lectura pero en funcion
--


In [35]:
def read(path):
    flag = 0
    #Columnas
    columns = 0
    #Total contenedores
    total = 0
    maxSize = 0
    stacks = []
    with open(path) as file:
        for line in file:
            s = line.split()
            #Si es la primera linea entonces se guarda las columnas y el total
            if flag == 0:
                #Actualiza columns
                columns = s[0]
                #Actualiza total
                total = s[1]
                #Actualiza flag para que no pase denuevo
                flag=1
            #En caso contrario sen otros datos de interes
            else:
                #Se ve el size mayor de la columna para definirlo automatico
                aux=int(s[0])
                if maxSize <= aux:
                    maxSize = aux;

                #Se crean los stacks
                temporalStack = []
                for i in range(1,len(s)):
                    temporalStack.append(int(s[i]))
                stacks.append(temporalStack)
    #Se crea el layout
    maxSize += 2
    layout = Layout.Layout(stacks, maxSize)
    
    return layout

#Test
#path="C:\\Users\\Leviathan\\Desktop\\trabajo\\Referencias\\BF\\Instancias\\10-6\\data10-6-1.dat"
#layout=read(path)
#print (layout.stacks)
#print (layout.H)

In [43]:
#Por conveniencias de la vida, tendre un field directo  a los stacks de layout
field=layout.stacks  
#print(field)

import copy 

#Se revisa una columna posible que nos generara la menor cantidad de problemas para ordenar
def select_col(layout):

    field=layout.stacks  
    best=-500
    col=-1
    
    for i in field:
        
        sort=copy.deepcopy(i)
        sort.sort(reverse=True)
        copia=copy.deepcopy(i)
        #print("#SEPARADOR------------------")
        
        #print(copia)
        #print(sort)
        
        #print("#SEPARADOR------------------")
        #Esto es para que no tome las columnas ordenadas
        if sort != copia:
            #Mientras no sean iguales
            while sort != copia:
                #Quitamos el superior
                sort.pop(len(sort)-1)
                copia.pop(len(copia)-1)

                if sort == copia:
                    #Se revisa que sea mejor que el anterior
                    if best < len(copia):
                        best=len(copia)
                        col=i
                    break
    #print(col)
    for i in range(0,len(field)-1):
        if layout.stacks[i] == col:
            return i
    #En caso que no queden columnas
    return -1

seleccionada=select_col(layout)
#print (seleccionada)
    

In [44]:
#Una vez esta seleccionada la columna se trabaja con ella, primero vemos la capacidad que tiene cada columna por separado
def capac(layout,column):
    field=layout.stacks
    capacidad=[]
    contador=0
    for i in field:
        if contador != column:
            capacidad.append(layout.H-len(i))
        else:
            capacidad.append(0)
        contador+=1
    return capacidad

print(seleccionada)
capacidad=capac(layout,seleccionada)
#print (field)
print (capacidad)


0
[0, 2, 2, 2, 2, 2]


In [45]:
#Se crea el field falso
stacks=[]

for i in range(0,len(capacidad)):
    stacks.append([])
print (stacks)

#Se crea donde guardar los movimientos (terminara siendo una lista de pares) seria un move[0] to move[1], move[2] to move[3] y asi
movs=[]




[[], [], [], [], [], []]


In [46]:
#Recopilacion de los datos importantes
seleccionada=select_col(layout)
#Capacidad maxima
capacidad=capac(layout,seleccionada)
#Columna
columna=copy.deepcopy(layout.stacks[seleccionada])
#Contenedores a retirar
retirar=remove_until(columna)
#Movimientos
movs=[]
stacks=[]
for i in range(0,len(capacidad)):
    stacks.append([])
#~°~°~°~°~°~°~°~°~°~~°~~°~~°~~°~REVISION SI TOMO LOS DATOS BIEN~°~°~°~°~°~°~°~°~°~°~°~°~°~°~°~°~°~°~°~°~°~°~°~°~°~°~°~°~
#~°~°~°~°~°~°~°~°~°~~°~~°~~°~~°~SEPARADOR MARCA FRUNA~°~°~°~°~°~°~°~°~°~°~°~°~°~°~°~°~°~°~°~°~°~°~°~°~°~°~°~°~
print(seleccionada)
print(retirar)
print(capacidad)
print(columna)
print(stacks)

#~°~°~°~°~°~°~°~°~°~~°~~°~~°~~°~SEPARADOR MARCA FRUNA~°~°~°~°~°~°~°~°~°~°~°~°~°~°~°~°~°~°~°~°~°~°~°~°~°~°~°~°~
#~°~°~°~°~°~°~°~°~°~~°~~°~~°~~°~FIN REVISION~°~°~°~°~°~°~°~°~°~°~°~°~°~°~°~°~°~°~°~°~°~°~°~°~°~°~°~°~

0
10
[0, 2, 2, 2, 2, 2]
[20, 27, 23, 54, 30, 28, 7, 48, 10, 36]
[[], [], [], [], [], []]


In [79]:
import Layout
import Greedy

def ranks(stack):
    stack.reverse()
    temp= {}
    r=1
    for n in stack:
        temp[n] = r
        r += 1
    return temp

def get_ranks(stack):
    r=1
    rank = {}
    for i in sorted(stack, reverse=True):
        rank[i] = r
        r += 1
    return rank

def fill_stack(layout, s_d, n, ori, rank):
    for i in range(n):
        s_o = Layout.select_origin_stack(layout, s_d, ori, rank)
        layout.move(s_o,s_d)

import numpy as np
def capacity(layout, s_o):
    capacity = 0
    for i in range(len(layout.stacks)):
        if i==s_o: continue
        capacity += layout.H - len(layout.stacks[i])
    return capacity            

def force_move(layout, s_o, pos, s_d): #pixie
    while pos<-1:
        s_tmp = Layout.select_destination_stack(layout, s_o, black_list=[s_d])
        layout.move(s_o,s_tmp)
        pos += 1
    return layout.move(s_o,s_d)

def search_highest(layout, c, ub, s_d):
    ret = None
    for pos in range(-1,-10,-1):
        max_c = 0
        for s in range(len(layout.stacks)):
            if s==s_d: continue
            ss = layout.stacks[s]
            if len(ss) < -pos: continue
            if ss[pos]>=c and ss[pos]<=ub and ss[pos]>max_c: 
                max_c=ss[pos]
                ret= (s,pos)
        if ret != None: return ret
    return None

In [80]:
#Finalmente vemos hasta donde se debe sacar de la columna que queremos arreglar
columna=layout.stacks[select_col(layout)]

#Retorna la cantidad de veces que debe sacarse un contenedor de la parte superior de la columna
def remove_until(column):
    sort=copy.deepcopy(column)
    sort.sort(reverse=True)
    copia=copy.deepcopy(column)
    
    contador=1
    while sort != copia:
        sort.pop(len(sort)-1)
        copia.pop(len(copia)-1)
        
        if sort == copia:
            return contador
        
        contador+=1

#test funcion
print (remove_until(columna))
print (columna)

10
[36, 10, 48, 7, 28, 30, 54, 23, 27, 20]


How to ocupar
----


In [89]:
#Creacion de layout
path="C:\\Users\\Leviathan\\Desktop\\trabajo\\Referencias\\BF\\Instancias\\10-6\\data10-6-2.dat"
layout=read(path)
#Test lectura
for s in layout.stacks:
    print (s)
print()

###SEPARADOR################################################

#seleccion de columna
seleccionada = select_col(layout)
#Test seleccion
print ("Seleccionada: "+str(seleccionada))

###SEPARADOR################################################

columna=copy.deepcopy(layout.stacks[seleccionada])
retirar = remove_until(columna)
#Test retirar
print("Se deben retirar: "+str(retirar)+" elementos")

###SEPARADOR################################################
#Mientras no este ordenado
#while columna != -1:
###SEPARADOR################################################  

rank = ranks(layout.stacks[seleccionada])
#Test ranks
print (rank)

###SEPARADOR################################################
def SDpp(layout, s_o, rank,quitar):
    capac = capacity(layout,s_o) # espacio libre
    ss_o = layout.stacks[s_o]
    
    print()
    print("THE DATA")
    print(capac)
    print(ss_o)
    print()
    
    while len(layout.stacks[s_o])>0:
        top = Layout.gvalue(ss_o)
        print ("TOP: "+str(top))
        
        slack = capac-len(ss_o) # holgura
        print ("SLACK: "+str(slack))
        #SEPARADOR############################
        print("\n instancia: \n")
        for s in layout.stacks:
            print (s)
        #SEPARADOR############################
        print("Columna seleccionada: ")
        print (s_o)
        #SEPARADOR############################
        max_pos=rank[top]+slack
        print(max_pos)
        s_d = Layout.select_destination_stack (layout, s_o, max_pos=rank[top]+slack)
        pos = layout.H - len(layout.stacks[s_d])
        if rank[top] < pos - slack:  # rellenar stack s_d
            #c=layout.stacks[s_o].pop(-1)
            fill_stack(layout, s_d, (pos - slack) - rank[top], s_o, rank)
            #layout.stacks[s_o].append(c)

        capac -= 1
        layout.move(s_o,s_d)
        
def SFpp(layout, s_d, rank, n):
    ub = 100
    cont = 0
    for r in rank:
        while True:
            (s, pos) = search_highest(layout, r, ub, s_d)
            
            if layout.sorted_elements[s] > len(layout.stacks[s])+pos: break #element is sorted
            c=force_move(layout, s, pos, s_d)
            ub=r
            cont+=1
            if cont == n: return
            if r==c: break
    while cont < n:
        Greedy.SF_move_d(layout, s_d)
        cont += 1

###SEPARADOR################################################      

SDpp(layout, seleccionada, rank,retirar)
for s in layout.stacks:
        print(s)
print("\n")
#Retirar cosas

###SEPARADOR################################################  
SFpp(layout,seleccionada,rank,retirar)
for s in layout.stacks:
        print(s)
print("\n")


[32, 2, 12, 1, 43, 35, 30, 57, 10, 6]
[22, 50, 25, 24, 40, 28, 41, 36, 56, 18]
[47, 42, 31, 13, 16, 58, 4, 44, 7, 19]
[5, 9, 51, 39, 20, 53, 15, 23, 27, 59]
[60, 14, 48, 33, 34, 38, 11, 46, 17, 49]
[54, 26, 21, 45, 55, 29, 8, 3, 52, 37]

Seleccionada: 4
Se deben retirar: 9 elementos
{49: 1, 17: 2, 46: 3, 11: 4, 38: 5, 34: 6, 33: 7, 48: 8, 14: 9, 60: 10}

THE DATA
10
[49, 17, 46, 11, 38, 34, 33, 48, 14, 60]

TOP: 60
SLACK: 0

 instancia: 

[32, 2, 12, 1, 43, 35, 30, 57, 10, 6]
[22, 50, 25, 24, 40, 28, 41, 36, 56, 18]
[47, 42, 31, 13, 16, 58, 4, 44, 7, 19]
[5, 9, 51, 39, 20, 53, 15, 23, 27, 59]
[49, 17, 46, 11, 38, 34, 33, 48, 14, 60]
[54, 26, 21, 45, 55, 29, 8, 3, 52, 37]
Columna seleccionada: 
4
10
TOP: 14
SLACK: 0

 instancia: 

[32, 2, 12, 1, 43, 35, 30, 57, 10, 6]
[22, 50, 25, 24, 40, 28, 41, 36, 56, 18]
[47, 42, 31, 13, 16, 58, 4, 44, 7, 19]
[5, 9, 51, 39, 20, 53, 15, 23, 27, 59, 60]
[49, 17, 46, 11, 38, 34, 33, 48, 14]
[54, 26, 21, 45, 55, 29, 8, 3, 52, 37]
Columna seleccionada: 


TypeError: cannot unpack non-iterable NoneType object